In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. 需求重要性

In [7]:
# 匯入套件
import numpy as np
import pandas as pd

from scipy.stats import pearsonr
from scipy.stats import kstest
from scipy.stats.contingency import association
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestRegressor

In [8]:
# 載入資料集 (final)
data = pd.read_excel("/content/drive/MyDrive/Python ipynb/Dataset/tb_cdri_work5_t3_final.xlsx")

In [9]:
# 載入資料集 (數據)
df = pd.read_excel("/content/drive/MyDrive/Python ipynb/Dataset/QFD數據_USB-C 充電器 未滿 60W.xlsx")
df.rename(columns = {'ASIN':'asin'}, inplace = True)
df.drop(df.columns[10:], axis=1, inplace=True)

In [10]:
# 刪除不要的欄位
data.drop(columns = ["產業", "Brand", "p_no", "name",
                    "index", "opinion_items_word", "review", "title",
                    "rating","date","p_url","level1_no","level2_no"], inplace=True)

In [11]:
# 篩選為60W以上
sixty_below = data[data["Category"] == "USB-C充電器(未滿60W)(USB-C Power adapter/charger Below 60W)"]

# 將emotion轉為 0:negative 1:positive
emotion_mapping = {'positive': 1, 'negative': 0}
sixty_below['emotion'] = sixty_below['emotion'].map(emotion_mapping)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# 針對主力商品 計算需求重要性
count_df = sixty_below["opinion_items"].value_counts().to_frame()
count_df = count_df.rename(columns={"opinion_items":"總評論數"})
count_df["重要性"] = count_df.apply(lambda r:r.rank(ascending=False)).astype(int)

In [13]:
# 計算平均
new_mean = sixty_below.groupby(by=["asin","opinion_items"])["emotion"].agg(["size","mean"]).reset_index()
new_mean.rename(columns = {"size":"產品評論總數","mean":"產品正評比例"}, inplace = True)

In [14]:
# 合併 df & new_mean datafeame
import warnings
warnings.filterwarnings("ignore")

final = pd.merge(df, new_mean, left_on= 'asin', right_on='asin')
final = final.drop(columns=["opinion_items"])

In [15]:
# 萃取 opinion_items 唯一值
items = list(sixty_below['opinion_items'].unique())

# 2. 計算相關性

In [16]:
feature_names = df.columns.tolist()[1:]

In [17]:
features = df.columns.size

Corr = np.empty((features-1, features-1))

for i in range(1, features):
    for j in range(1, features):
        A = df.iloc[:, i]
        B = df.iloc[:, j]
        if (df.iloc[:, i].dtype == 'float64') and (df.iloc[:, j].dtype == 'float64'):
            Corr[i-1, j-1] = pearsonr(A.values.T, B.values.T)[0]
        else:
            C = pd.crosstab(A, B).values       
            Corr[i-1, j-1] = association(C, method = 'cramer', correction = True)

corr = pd.DataFrame(Corr, index = feature_names, columns = feature_names)
corr

,尺寸(面積),重量,特徵,USB埠數量,總輸出功率,輸出,快充協議,認證,保固
尺寸(面積),1.000000,0.980196,0.969918,1.000000,1.000000,1.000000,0.968644,1.000000,0.981071
重量,0.980196,1.000000,0.963624,1.000000,0.973865,0.984604,0.937392,1.000000,0.948958
特徵,0.969918,0.963624,1.000000,1.000000,0.946485,0.967169,1.000000,1.000000,0.960143
USB埠數量,1.000000,1.000000,1.000000,1.000000,0.796099,1.000000,0.658975,1.000000,0.557247
總輸出功率,1.000000,0.973865,0.946485,0.796099,1.000000,1.000000,0.677589,0.692219,0.457331
輸出,1.000000,0.984604,0.967169,1.000000,1.000000,1.000000,0.705976,0.790569,0.669570
快充協議,0.968644,0.937392,1.000000,0.658975,0.677589,0.705976,1.000000,0.889757,0.714718
認證,1.000000,1.000000,1.000000,1.000000,0.692219,0.790569,0.889757,1.000000,0.645497
保固,0.981071,0.948958,0.960143,0.557247,0.457331,0.669570,0.714718,0.645497,1.000000


# 3. QFD數據

In [18]:
# 載入資料集 (數據)
df = pd.read_excel("/content/drive/MyDrive/Python ipynb/Dataset/QFD數據_USB-C 充電器 未滿 60W.xlsx")
df.rename(columns = {'ASIN':'asin'}, inplace = True)
df.drop(df.columns[10:], axis=1, inplace=True)

### 1.將重量：去掉單位 ounces

In [19]:
df["重量"] = df["重量"].str.split(" ", expand=True)[0]
df["總輸出功率"] = df["總輸出功率"].str.split("W", expand=True)[0]

### 2.USB埠數量: 要切割成 2 欄，分別代表 USB-A 和 USB-C的數量

In [20]:
USB_C = []
USB_A = []

for i in df["USB埠數量"]:
    if len(i)//2 == 1:
        USB_C.append(i[::])
        x = i + "0A"
        y = x[2:]
        USB_A.append(y)
    else:
        C = slice(0, len(i)//2)
        A = slice(len(i)//2, len(i))
        USB_C.append(i[C]), USB_A.append(i[A])
        
        
# 把Ｃ替換成空白，只保留數值
USBC = [i.replace("C", "") for i in USB_C]
# 把A替換成空白，只保留數值
USBA = [i.replace("A", "") for i in USB_A]


# 新增新的兩個欄位USB-C,USB-A，並刪除欄位USB埠數量
df["USB-C數量"] = pd.DataFrame(USBC)
df["USB-A數量"] = pd.DataFrame(USBA)
df = df.drop(columns=["USB埠數量"], axis=1)

### 3.輸出: 類似作法分割成 2 columns，分別代表 USB-A 和 USB-C的最高輸出功率

In [21]:
# Import Library Regex
import re

# Define Function for Usb-A & Usb-C
def Extract_Usb_Max(input):
    numbers = max(map(float, re.findall('\d*\.\d+|\d+', input)))
    return numbers

def UsbC_Max(amount, type):
    for i in range(len(df)):
        if df[amount][i] == '1':
            type.append(Extract_Usb_Max(df["輸出"][i][:10]))
        
        if df[amount][i] == '2':
            type.append(Extract_Usb_Max(df["輸出"][i][:21]))
        
        if df[amount][i] == '3':
            type.append(Extract_Usb_Max(df["輸出"][i][:37]))
        
        if df[amount][i] == '4':
            type.append(Extract_Usb_Max(df["輸出"][i][:58]))

def UsbA_Max(amount, type):
    for i in range(len(df)):
        if df[amount][i] == '0':
            type.append(0)

        if df[amount][i] == '1':
            type.append(Extract_Usb_Max(df["輸出"][i][-10:]))
        
        if df[amount][i] == '2':
            type.append(Extract_Usb_Max(df["輸出"][i][-21:]))
        
        if df[amount][i] == '3':
            type.append(Extract_Usb_Max(df["輸出"][i][-32:]))
        
        if df[amount][i] == '4':
            type.append(Extract_Usb_Max(df["輸出"][i][-50:]))
            


# Execute Function & Get the Final Dataframe
numberC = []
numberA = []
UsbC_Max('USB-C數量', numberC)
UsbA_Max('USB-A數量', numberA)

df['USB-C Max'] = numberC
df['USB-A Max'] = numberA

### 4.“特徵”、“快充協議”和“認證”: 則可以看成是 multi-label 的問題 

In [22]:
# 匯入套件
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [23]:
# 特徵欄位處理
df_final = df.join(pd.DataFrame(mlb.fit_transform([str(df.loc[i,'特徵']).split("\n")
                                                   for i in range(len(df))]), columns=mlb.classes_))
col_特徵 = mlb.classes_.tolist()
col_特徵.remove('nan')
df_final = df_final.drop("nan", axis=1)

In [24]:
# 快充協議欄位處理
df_final = df_final.join(pd.DataFrame(mlb.fit_transform([str(df.loc[i,'快充協議']).split("\n")
                                                   for i in range(len(df))]), columns=mlb.classes_))
col_快充 = mlb.classes_.tolist()
col_快充.remove('nan')
df_final = df_final.drop("nan", axis=1)

In [25]:
# 認證欄位處理
df_final = df_final.join(pd.DataFrame(mlb.fit_transform([str(df.loc[i,'認證']).split("\n")
                                                   for i in range(len(df))]), columns=mlb.classes_))
col_認證 = mlb.classes_.tolist()
col_認證.remove('nan')
df_final = df_final.drop("nan", axis=1)

In [26]:
# 改變資料型態為浮點數
df1 = df_final.astype({'重量':'float', 
                       "USB-C數量":"float",
                       "USB-A數量":"float", 
                       "總輸出功率":"float"})

In [27]:
# 刪除原始數據欄位
df = df1.drop(columns={'特徵', "輸出", "快充協議", "認證"}, axis=1)

In [28]:
spec_columns = df.columns.tolist()[1:]

## 3. 合併及建模

In [29]:
# 合併 data & new_mean datafeame
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

importances = np.empty((0, 40), int)

number_list = [i for i in range(10, 210, 10)] # 設定 RandomForestRegression 模型裡的 n_estimaters 參數

mse_list = []                 # 建立MSE串列
maxd_list = [1, 2, 3]         # 建立樹的深度
eta_list = [0.01, 0.02, 0.03] # 建立學習率

for i in items:
    comment_temp = new_mean[new_mean['opinion_items'] == i].reset_index(drop = True)
    spec_comment = pd.merge(df, comment_temp, left_on= 'asin', right_on='asin')
    spec_comment.drop(columns = ['opinion_items'], axis=1, inplace=True)

    if len(spec_comment) > 2:
        X = spec_comment.iloc[:,1:-2].values
        y = spec_comment.iloc[:,-1].values
    
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    
    # Fit the model
    best_mse = 1
    best_par = {}
    for n in number_list:
        for d in maxd_list:
            for eta in eta_list:
                xgbModel = xgb.XGBRegressor(n_estimators = n, max_depth = d, objectvie='reg:squarederror', 
                                        booster='gbtree', learning_rate = eta, verbosity = 2)
                xgbModel.fit(X_train, y_train)
                predicted = xgbModel.predict(X_test)
                
                mse = cross_val_score(xgbModel, X_train, y_train, cv=len(y_train), scoring='neg_mean_squared_error').mean()           
                if mse < best_mse:
                    best_mse = mse
                    best_par = {'number':n, 'max_depth':d, "eta":eta}
    
    xgbModel = xgb.XGBRegressor(n_estimators = best_par['number'], max_depth=best_par["max_depth"], objectvie='reg:squarederror', 
                                booster='gbtree', learning_rate=best_par["eta"], verbosity = 2)
    xgbModel.fit(X_train, y_train)
    mse = metrics.mean_squared_error(y_test, predicted) 
    mse_list.append([mse, best_par])

    importances = np.vstack([importances, xgbModel.feature_importances_])
    
MSE = pd.DataFrame(mse_list, columns=["MSE Score", "Best parameters"])
MSE

串流輸出內容已截斷至最後 5000 行。
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12:55:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots

,MSE Score,Best parameters
0,0.019816,"{'number': 10, 'max_depth': 3, 'eta': 0.01}"
1,0.064772,"{'number': 200, 'max_depth': 1, 'eta': 0.03}"
2,0.011080,"{'number': 10, 'max_depth': 1, 'eta': 0.01}"
3,0.014953,"{'number': 200, 'max_depth': 1, 'eta': 0.03}"
4,0.373284,"{'number': 200, 'max_depth': 2, 'eta': 0.03}"
5,0.381518,"{'number': 200, 'max_depth': 3, 'eta': 0.03}"
6,0.027002,"{'number': 10, 'max_depth': 2, 'eta': 0.01}"
7,0.038958,"{'number': 10, 'max_depth': 1, 'eta': 0.01}"
8,0.074168,"{'number': 200, 'max_depth': 2, 'eta': 0.03}"
9,0.184511,"{'number': 200, 'max_depth': 3, 'eta': 0.03}"


In [30]:
df = pd.DataFrame(importances, index = items, columns = spec_columns)

In [31]:
sum_col_特徵 = df[col_特徵].sum(axis = 1)
df1 = df.assign(特徵 = sum_col_特徵)
df = df1.drop(columns = col_特徵)

sum_col_快充 = df[col_快充].sum(axis = 1)
df2 = df.assign(快充 = sum_col_快充)
df = df2.drop(columns = col_快充)

sum_col_認證 = df[col_認證].sum(axis = 1)
df3 = df.assign(認證 = sum_col_認證)
df = df3.drop(columns = col_認證)

# 將 df 匯出成Excel
df.to_excel("/content/drive/MyDrive/Python ipynb/Excel檔/USB-C_充電器_60W以下訓練XGBT.xlsx")